In [1]:
import pandas as pd
import sqlite3
import datetime
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

from sqlalchemy import create_engine

In [2]:
#connection to a PostgreSQL local database
# ===========================================

# user_name_var = input('What is the username of your database? ')
host_var = input('What is the host address of your database?')
pw = input("Enter the database password: ")
# db_name = input("Enter the name of the database: ")

string_output = "postgresql://postgres"+':'+pw+'@'+host_var+":5432"+'/earthquake_db'
engine = create_engine(string_output)
connection = engine.connect()
print("Connected to database...")

Connected to database...


In [3]:
#create and connect to a sqlite3 db
conn = sqlite3.connect(r'/Users/rosaicelaroman/Desktop/portfolio_work/earthquake_modeling/Database/db/earthquake_db')

Adding 1 month of data

In [5]:
page_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

df = pd.read_csv(page_url)
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-04-01T05:24:06.180Z,38.822166,-122.808502,2.69,0.96,md,19.0,38.00,0.001736,0.0200,...,2022-04-01T05:25:43.196Z,"7km NW of The Geysers, CA",earthquake,0.24,0.54,0.070000,4.0,automatic,nc,nc
1,2022-04-01T05:18:15.843Z,38.226300,-117.863400,2.00,1.30,ml,14.0,139.64,0.099000,0.2509,...,2022-04-01T05:21:59.407Z,"28 km SE of Mina, Nevada",earthquake,NaN,1.60,0.330000,8.0,automatic,nn,nn
2,2022-04-01T05:15:47.130Z,32.806500,-115.460833,6.86,1.97,ml,21.0,90.00,0.011890,0.3100,...,2022-04-01T05:19:30.881Z,"8km W of Holtville, CA",earthquake,0.69,0.86,0.171000,26.0,automatic,ci,ci
3,2022-04-01T05:15:20.400Z,32.802667,-115.455167,7.29,1.54,ml,13.0,139.00,0.007535,0.1200,...,2022-04-01T05:19:49.058Z,"7km W of Holtville, CA",earthquake,0.48,0.46,0.164000,15.0,automatic,ci,ci
4,2022-04-01T05:14:46.410Z,32.743167,-115.539667,11.05,1.52,ml,7.0,181.00,0.099820,0.2300,...,2022-04-01T05:18:29.328Z,"2km NW of Heber, CA",earthquake,1.10,1.45,0.170000,17.0,automatic,ci,ci
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9354,2022-03-02T06:21:30.517Z,67.324800,-156.206700,1.30,1.10,ml,NaN,NaN,NaN,0.7200,...,2022-03-02T17:12:56.305Z,"55 km NNE of Kobuk, Alaska",earthquake,NaN,0.70,NaN,NaN,reviewed,ak,ak
9355,2022-03-02T06:17:34.572Z,62.557300,-151.282900,88.60,1.40,ml,NaN,NaN,NaN,0.3700,...,2022-03-02T06:20:43.153Z,"27 km WNW of Petersville, Alaska",earthquake,NaN,1.10,NaN,NaN,automatic,ak,ak
9356,2022-03-02T06:12:25.443Z,61.957200,-149.062300,27.20,1.90,ml,NaN,NaN,NaN,0.8200,...,2022-03-02T06:23:23.541Z,Southern Alaska,earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
9357,2022-03-02T06:01:04.250Z,36.049833,-120.614833,2.83,1.11,md,26.0,80.00,0.024220,0.0600,...,2022-03-02T09:41:11.580Z,"24km NW of Parkfield, CA",earthquake,0.24,0.55,0.194000,15.0,reviewed,nc,nc


In [6]:
#append the new data from USGS to both databases
#=============
#PostgreSQL
df.to_sql('earthquake_raw', engine, if_exists='append', index = False)
#=============
#SQLite
cur = conn.cursor()
df.to_sql('earthquake_raw', conn, if_exists='append', index=False)

9359

Load from database to update with new rows and remove duplicate rows

In [7]:
#Dataframe from postgresql
model_df = pd.read_sql_table(
    "earthquake_raw",
    con=engine
)
model_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.87500,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290000,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.11900,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460000,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48140,2022-03-02T06:21:30.517Z,67.324800,-156.206700,1.30,1.10,ml,NaN,NaN,NaN,0.7200,...,2022-03-02T17:12:56.305Z,"55 km NNE of Kobuk, Alaska",earthquake,NaN,0.70,NaN,NaN,reviewed,ak,ak
48141,2022-03-02T06:17:34.572Z,62.557300,-151.282900,88.60,1.40,ml,NaN,NaN,NaN,0.3700,...,2022-03-02T06:20:43.153Z,"27 km WNW of Petersville, Alaska",earthquake,NaN,1.10,NaN,NaN,automatic,ak,ak
48142,2022-03-02T06:12:25.443Z,61.957200,-149.062300,27.20,1.90,ml,NaN,NaN,NaN,0.8200,...,2022-03-02T06:23:23.541Z,Southern Alaska,earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
48143,2022-03-02T06:01:04.250Z,36.049833,-120.614833,2.83,1.11,md,26.0,80.00,0.02422,0.0600,...,2022-03-02T09:41:11.580Z,"24km NW of Parkfield, CA",earthquake,0.24,0.55,0.194000,15.0,reviewed,nc,nc


In [8]:
#dataframe from SQLite
model_df2 = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
model_df2

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.87500,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.290000,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.11900,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.460000,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48140,2022-03-02T06:21:30.517Z,67.324800,-156.206700,1.30,1.10,ml,NaN,NaN,NaN,0.7200,...,2022-03-02T17:12:56.305Z,"55 km NNE of Kobuk, Alaska",earthquake,NaN,0.70,NaN,NaN,reviewed,ak,ak
48141,2022-03-02T06:17:34.572Z,62.557300,-151.282900,88.60,1.40,ml,NaN,NaN,NaN,0.3700,...,2022-03-02T06:20:43.153Z,"27 km WNW of Petersville, Alaska",earthquake,NaN,1.10,NaN,NaN,automatic,ak,ak
48142,2022-03-02T06:12:25.443Z,61.957200,-149.062300,27.20,1.90,ml,NaN,NaN,NaN,0.8200,...,2022-03-02T06:23:23.541Z,Southern Alaska,earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
48143,2022-03-02T06:01:04.250Z,36.049833,-120.614833,2.83,1.11,md,26.0,80.00,0.02422,0.0600,...,2022-03-02T09:41:11.580Z,"24km NW of Parkfield, CA",earthquake,0.24,0.55,0.194000,15.0,reviewed,nc,nc


In [9]:
#check for duplicates in postgres data
boolean = model_df.duplicated(subset=['id']).any()
print(boolean)

True


In [10]:
#check for duplicates in sqlite data
boolean = model_df2.duplicated(subset=['id']).any()
print(boolean)

True


In [11]:
#drop duplicates in postgres data
model_df.drop_duplicates(subset=['id'], inplace=True)
print(model_df)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
38799  2022-04-01T04:48:08.310Z  38.814335 -122.792664   -0.17  0.84      md   
38800  2022-04-01T04:40:57.610Z  38.821167 -122.788170    1.45  1.14      md   
38801  2022-04-01T04:21:57.850Z  36.900501 -120.989502    5.11  2.29      md   
38803  2022-04-01T04:05:26.545Z -10.369600  -78.346100   66.91  4.30      mb   
38804  2022-04-01T03:54:45.620Z  17.932167  -66.709167    8.50  2.08      md   

        nst     gap      dmin     rms  

In [12]:
#drop duplicates in sqlite data
model_df2.drop_duplicates(subset=['id'], inplace=True)
print(model_df2)

                           time   latitude   longitude   depth   mag magType  \
0      2022-01-06T01:42:39.855Z  37.384000 -117.107500    1.10  1.30      ml   
1      2022-01-06T01:31:33.698Z  38.181000 -117.818000    7.30  1.40      ml   
2      2022-01-06T01:24:20.083Z  61.340700 -149.885200   30.00  1.50      ml   
3      2022-01-06T01:15:15.851Z  62.870300 -150.762500  111.30  2.10      ml   
4      2022-01-06T01:10:44.629Z  64.014500 -148.765700    1.90  1.50      ml   
...                         ...        ...         ...     ...   ...     ...   
38792  2022-04-01T04:48:08.310Z  38.814335 -122.792664   -0.17  0.84      md   
38793  2022-04-01T04:40:57.610Z  38.821167 -122.788170    1.45  1.14      md   
38794  2022-04-01T04:21:57.850Z  36.900501 -120.989502    5.11  2.29      md   
38796  2022-04-01T04:05:26.545Z -10.369600  -78.346100   66.91  4.30      mb   
38797  2022-04-01T03:54:45.620Z  17.932167  -66.709167    8.50  2.08      md   

        nst     gap      dmin     rms  

In [13]:
#replace the table in postgres with one without duplicates
model_df.to_sql('earthquake_raw', engine, if_exists='replace', index = False)

797

In [14]:
#replace the table in sqlite with one without duplicates
model_df2.to_sql('earthquake_raw', conn, if_exists='replace', index = False)

38797

In [15]:
#copy data as csv as backup method
model_df.to_csv("../Database/csv/earthquake_raw.csv",index=False)

In [4]:
# Import API key
import sys
weather_api_key = "YOUR KEY HERE!"

if weather_api_key == "YOUR KEY HERE!":
    sys.path.append(r"/Users/rosaicelaroman/Desktop/Data_BootCamp/APIs/config_keys/")
    from api_keys_v2 import weather_api_key

In [17]:
#create new dataframe for locatiions
# location_df = pd.read_sql_query("SELECT * FROM earthquake_raw", conn)
# location_df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2022-01-06T01:42:39.855Z,37.384000,-117.107500,1.10,1.30,ml,8.0,149.46,0.875000,0.2175,...,2022-01-06T01:47:19.317Z,"37 km SSE of Goldfield, Nevada",earthquake,NaN,39.10,0.29000,4.0,automatic,nn,nn
1,2022-01-06T01:31:33.698Z,38.181000,-117.818000,7.30,1.40,ml,11.0,154.64,0.119000,0.1017,...,2022-01-06T01:47:40.245Z,"34 km SE of Mina, Nevada",earthquake,NaN,2.00,0.46000,5.0,automatic,nn,nn
2,2022-01-06T01:24:20.083Z,61.340700,-149.885200,30.00,1.50,ml,NaN,NaN,NaN,0.7500,...,2022-01-06T01:29:22.015Z,"5 km ESE of Point MacKenzie, Alaska",earthquake,NaN,0.20,NaN,NaN,automatic,ak,ak
3,2022-01-06T01:15:15.851Z,62.870300,-150.762500,111.30,2.10,ml,NaN,NaN,NaN,0.3200,...,2022-01-06T01:29:21.875Z,"41 km N of Petersville, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2022-01-06T01:10:44.629Z,64.014500,-148.765700,1.90,1.50,ml,NaN,NaN,NaN,0.7200,...,2022-01-06T01:15:56.292Z,"17 km E of Ferry, Alaska",earthquake,NaN,0.60,NaN,NaN,automatic,ak,ak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38792,2022-04-01T04:48:08.310Z,38.814335,-122.792664,-0.17,0.84,md,9.0,74.00,0.006684,0.0200,...,2022-04-01T05:18:12.579Z,"5km NW of The Geysers, CA",earthquake,0.26,0.85,NaN,1.0,automatic,nc,nc
38793,2022-04-01T04:40:57.610Z,38.821167,-122.788170,1.45,1.14,md,17.0,37.00,0.009826,0.0300,...,2022-04-01T05:25:10.615Z,"6km NNW of The Geysers, CA",earthquake,0.25,0.40,0.49000,3.0,automatic,nc,nc
38794,2022-04-01T04:21:57.850Z,36.900501,-120.989502,5.11,2.29,md,39.0,156.00,0.141500,0.1500,...,2022-04-01T05:33:12.669Z,"22km SW of Los Banos, CA",earthquake,0.51,1.29,0.12000,36.0,automatic,nc,nc
38795,2022-04-01T04:05:26.545Z,-10.369600,-78.346100,66.91,4.30,mb,NaN,158.00,2.182000,0.9400,...,2022-04-01T04:31:09.040Z,near the coast of central Peru,earthquake,11.90,8.80,0.11900,22.0,reviewed,us,us


In [18]:
#create new df with only necessary columns
# cleaned_df = location_df.drop(columns=['nst','rms','magNst','net', 'time', 'magType', 'gap','dmin', 'updated', 'place', 'type', 'horizontalError', 'depthError', 'magError', 'status', 'locationSource', 'magSource'])
# cleaned_df

,latitude,longitude,depth,mag,id
0,37.384000,-117.107500,1.10,1.30,nn00831163
1,38.181000,-117.818000,7.30,1.40,nn00831161
2,61.340700,-149.885200,30.00,1.50,ak0229xypaw
3,62.870300,-150.762500,111.30,2.10,ak0229xwriz
4,64.014500,-148.765700,1.90,1.50,ak0229xvr66
...,...,...,...,...,...
38792,38.814335,-122.792664,-0.17,0.84,nc73712961
38793,38.821167,-122.788170,1.45,1.14,nc73712951
38794,36.900501,-120.989502,5.11,2.29,nc73712941
38795,-10.369600,-78.346100,66.91,4.30,us7000gyvi


http://api.openweathermap.org/geo/1.0/reverse?lat={lat}&lon={lon}&limit={limit}&appid={API key}

In [19]:
#create new columns for data from API
# cleaned_df['Location Name']=""
# cleaned_df['City']=""
# cleaned_df['State']=""
# cleaned_df['Country']=""
# cleaned_df

,latitude,longitude,depth,mag,id,Location Name,City,State,Country
0,37.384000,-117.107500,1.10,1.30,nn00831163,,,,
1,38.181000,-117.818000,7.30,1.40,nn00831161,,,,
2,61.340700,-149.885200,30.00,1.50,ak0229xypaw,,,,
3,62.870300,-150.762500,111.30,2.10,ak0229xwriz,,,,
4,64.014500,-148.765700,1.90,1.50,ak0229xvr66,,,,
...,...,...,...,...,...,...,...,...,...
38792,38.814335,-122.792664,-0.17,0.84,nc73712961,,,,
38793,38.821167,-122.788170,1.45,1.14,nc73712951,,,,
38794,36.900501,-120.989502,5.11,2.29,nc73712941,,,,
38795,-10.369600,-78.346100,66.91,4.30,us7000gyvi,,,,


In [5]:
location_df = pd.read_sql_query("SELECT * FROM earthquake_raw_locations", conn)
location_df

,latitude,longitude,depth,mag,id,Location Name,City,State,Country
0,37.384000,-117.107500,1.10,1.30,nn00831163,Nye County,Nye County,Nevada,US
1,38.181000,-117.818000,7.30,1.40,nn00831161,Esmeralda County,Esmeralda County,Nevada,US
2,61.340700,-149.885200,30.00,1.50,ak0229xypaw,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
3,62.870300,-150.762500,111.30,2.10,ak0229xwriz,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
4,64.014500,-148.765700,1.90,1.50,ak0229xvr66,Denali,Denali,Alaska,US
...,...,...,...,...,...,...,...,...,...
38781,35.322667,-117.783333,4.70,1.09,ci39983311,,,,
38782,63.390200,-151.341800,0.00,1.80,ak02246jyds7,,,,
38783,60.602200,-150.643000,19.40,2.10,ak02246jxbq8,,,,
38784,35.965500,-117.677167,2.37,0.79,ci39983287,,,,


In [6]:
base_url = "http://api.openweathermap.org/geo/1.0/reverse?"



http://api.openweathermap.org/geo/1.0/reverse?lat=51.5098&lon=-0.1180&limit=5&appid={API key}

In [24]:
response = requests.get(base_url+"lat="+'37.3840'+'&lon='+'-117.107500'+'&limit='+'1'+'&appid=' + weather_api_key).json()
print(response[0]['country'])

US


In [7]:
import requests
import time

for index, row in location_df.iterrows():
    # count+=1
    lat = row['latitude']
    lon = row['longitude']
    loc_found = []

    if row['Country'] != "":
        continue

    print(f'Retrieving results for Index {index}: {lat}')
    try:
        response = requests.get(base_url+"lat="+str(lat)+'&lon='+str(lon)+'&limit='+'1'+'&appid=' + weather_api_key).json()
        # response = requests.get(base_url+city+"&units="+units+'&appid=' + weather_api_key).json()
        location_df.loc[index, 'Location Name'] = response[0]['name']
        location_df.loc[index, 'City'] = response[0]['local_names']['en']
        location_df.loc[index, 'State'] = response[0]['state']
        location_df.loc[index, 'Country'] = response[0]['country']
        loc_found.append(lat)

        print(f'Found lat: {lat}')
    except:
        print(f"Missing lat: {lat}... Skipping")
        location_df.loc[index, 'Location Name'] = float('NaN')
        location_df.loc[index, 'City'] = float('NaN')
        location_df.loc[index, 'State'] = float('NaN')
        location_df.loc[index, 'Country'] = float('NaN')
    print(f'--------------------')
    time.sleep(2)


Retrieving results for Index 16135: 19.482
Found lat: 19.482
--------------------
Retrieving results for Index 16136: 31.0613
Found lat: 31.0613
--------------------
Retrieving results for Index 16137: 38.8276672
Found lat: 38.8276672
--------------------
Retrieving results for Index 16138: 38.7965012
Found lat: 38.7965012
--------------------
Retrieving results for Index 16139: 52.168
Found lat: 52.168
--------------------
Retrieving results for Index 16140: 19.48
Found lat: 19.48
--------------------
Retrieving results for Index 16141: 19.2211666666667
Found lat: 19.2211666666667
--------------------
Retrieving results for Index 16142: 36.008
Found lat: 36.008
--------------------
Retrieving results for Index 16143: 38.8328323
Found lat: 38.8328323
--------------------
Retrieving results for Index 16144: 32.7178333
Found lat: 32.7178333
--------------------
Retrieving results for Index 16145: 19.4825
Found lat: 19.4825
--------------------
Retrieving results for Index 16146: 36.8225


KeyboardInterrupt: 

In [8]:
location_df

,latitude,longitude,depth,mag,id,Location Name,City,State,Country
0,37.384000,-117.107500,1.10,1.30,nn00831163,Nye County,Nye County,Nevada,US
1,38.181000,-117.818000,7.30,1.40,nn00831161,Esmeralda County,Esmeralda County,Nevada,US
2,61.340700,-149.885200,30.00,1.50,ak0229xypaw,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
3,62.870300,-150.762500,111.30,2.10,ak0229xwriz,Matanuska-Susitna,Matanuska-Susitna,Alaska,US
4,64.014500,-148.765700,1.90,1.50,ak0229xvr66,Denali,Denali,Alaska,US
...,...,...,...,...,...,...,...,...,...
38781,35.322667,-117.783333,4.70,1.09,ci39983311,,,,
38782,63.390200,-151.341800,0.00,1.80,ak02246jyds7,,,,
38783,60.602200,-150.643000,19.40,2.10,ak02246jxbq8,,,,
38784,35.965500,-117.677167,2.37,0.79,ci39983287,,,,


In [9]:
#replace the table in sqlite with one without duplicates
location_df.to_sql('earthquake_raw_locations', conn, if_exists='replace', index = False)

#replace the table in postgres with one without duplicates
location_df.to_sql('earthquake_raw_locations', engine, if_exists='replace', index = False)

786

In [28]:
# Close connection to SQLite database
conn.close()
connection.close()